### The `Image` Widget

This widget can be used to display images given in the form of base64 encoded `Text`. The widget has a `B64Value` field, which can be changed to display images to it. It also has an `ImageFormat` field, which is set to `PNG` by default.

In [21]:
:t IHaskell.Display.base64
:t IHaskell.Display.encode64

IHaskell.Display.base64 :: ByteString -> Base64

IHaskell.Display.encode64 :: String -> Base64

The following example downloads an xkcd comic and displays it in an image widget.

In [4]:
-- Uncomment the line below to install HTTP if you don't have it
-- :!cabal install HTTP
import Network.HTTP
import IHaskell.Display (encode64)

get url = simpleHTTP (getRequest url) >>= getResponseBody
jpg <- get "http://imgs.xkcd.com/comics/functional.png"

img <- mkImageWidget
setField img SB64Value (encode64 jpg)
img

Replace the call to undefined by the path to an image, and it will be displayed in an image widget.

In [5]:
imgpath = undefined

import qualified Data.ByteString as B
import IHaskell.Display (encode64)

i <- mkImageWidget
B.readFile imgpath >>= setField i SB64Value . encode64

Couldn't match type ‘String’ with ‘B.ByteString’
Expected type: B.ByteString -> IHaskell.Display.Base64
  Actual type: String -> IHaskell.Display.Base64
In the second argument of ‘(.)’, namely ‘encode64’
In the second argument of ‘(>>=)’, namely ‘setField i SB64Value . encode64’

### The `Selection` Widgets

These widgets can be used to select one from many. The `SelectMultiple` widget allows multiple selections, whereas `Dropdown`, `RadioButtons`, `ToggleButtons`, and `Select` only allow one selection.

In [23]:
-- Allows single selection
tgbs <- mkToggleButtons

-- Allows multiple selections
msel <- mkSelectMultiple

In [24]:
setField msel SDescription "Functions to show (One or more)"
setField msel SOptions (OptionLabels ["sin", "cos"])

setField tgbs SDescription "Plot style"
setField tgbs SOptions (OptionLabels ["line", "point"])

In [25]:
import Graphics.Rendering.Chart.Easy hiding (tan)
import Graphics.Rendering.Chart.Backend.Cairo
import qualified Data.ByteString as B

import Control.Monad (when, forM)
import Data.Maybe (fromJust)

dset :: [(String, [(Double, Double)])]
dset = [("sin", zmap sin r), ("cos", zmap cos r)]
  where zmap f xs = zip xs (map f xs)
        r = [0, 0.1 .. 6.3]

i <- mkImageWidget
setField i SWidth 500
setField i SHeight 500

-- Redraw the plot based on values from the widgets
refresh = do
  -- Read values from the widgets
  funs <- map unpack <$> getField msel SSelectedValues
  sty <- unpack <$> getField tgbs SSelectedValue
  
  let pts = zip funs (map (fromJust . flip lookup dset) funs)
      opts = def { _fo_size = (500, 500) }
  toFile opts ".chart" $ do
    layout_title .= "Plotting: " ++ unwords funs
    if sty == "line"
      then mapM_ (\(s, ps) -> plot (line s [ps])) pts
      else mapM_ (\(s, ps) -> plot (points s ps)) pts

  img <- B.readFile ".chart"
  setField i SB64Value (base64 img)
  
-- Add event handlers to make widgets work
setField msel SSelectionHandler refresh
setField tgbs SSelectionHandler refresh

In [26]:
-- Display the widgets
msel
tgbs
i

The `Dropdown`, `RadioButtons` and `Select` widgets behave just like the `ToggleButtons` widget. They have the same properties, and the same functionality.

### The Numeric Widgets

**NOTE**: The following examples use widgets with `Int` in their names. There are also analogous widgets with `Float` in their names.

As the widgets are the same operationally, only the `Int` widgets are shown.

#### `IntText` and `BoundedIntText`

In [27]:
int <- mkIntText
bit <- mkBoundedIntText

In [28]:
int
bit

Both the widgets are similar, but the second one possesses some additional properties.

In [29]:
setField bit SMaxInt 20
setField bit SMinInt 10
setField bit SChangeHandler (getField bit SIntValue >>= print)

Now, the first widget will accept arbitrary input whereas the second one wil accept input the the 10-20 range. For example, try entering large values and hitting return/enter in the second widget.

#### `IntSlider` and `IntRangeSlider`

Both these widgets are sliders (duh!). `IntSlider` represents a single value, whereas `IntRangeSlider` represents a pair (range) of values.

In [30]:
ins <- mkIntSlider
irs <- mkIntRangeSlider

In [31]:
ins
irs

In [32]:
getField irs SIntPairValue

(25,75)

#### `IntProgress`

This widget is meant to be used as a progress bar.

In [33]:
inp <- mkIntProgress
inp

In [34]:
setField inp SIntValue 42